In [1]:
from sklearn.linear_model import LogisticRegression
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np

In [2]:
df= pd.read_excel('Dimension_Assigned_Data.xlsx')
df = df.rename(columns={'Manual_Polarity':'Label'})

In [3]:
df.shape

(685, 8)

In [4]:
type(df['Label'][0])

numpy.int64

In [5]:
##converting '999' to 0
#def con(val):
#    for i in val:
#        if i==999:
#            val=0
#    return val        

In [6]:
df['Label']=np.where(df['Label']==999,0,df['Label'])

In [7]:
X = df.drop(['Label'], axis=1)

In [8]:
y=df['Label']

In [9]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, HashingVectorizer

In [10]:
corpus=[]
for i in X['Review']:
    corpus.append(i)

In [11]:
#cv = CountVectorizer(max_features=200,ngram_range=(2,4))
#X = cv.fit_transform(corpus).toarray() 

### TF-IDF 

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer
tf = TfidfVectorizer()
X = tf.fit_transform(corpus).toarray()

In [13]:
## Train Test Split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=0)

In [14]:
X_train

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

### Over-Sampling

In [15]:
from imblearn.over_sampling import SMOTE
from collections import Counter
oversample = SMOTE()
X_train1, y_train1 = oversample.fit_resample(X_train, y_train)
from collections import Counter
print("The number of classes before fit {}".format(Counter(y_train)))
print("The number of classes after fit {}".format(Counter(y_train1)))

The number of classes before fit Counter({1: 262, 0: 115, -1: 81})
The number of classes after fit Counter({0: 262, 1: 262, -1: 262})


### One vs rest Logistic Regression

In [16]:
# Create one-vs-rest logistic regression object
clf = LogisticRegression(random_state=0, multi_class='ovr')

In [17]:
model = clf.fit(X_train1, y_train1)

In [18]:
pred = model.predict(X_test)

In [19]:
from sklearn import metrics
score = metrics.accuracy_score(y_test, pred)
score

0.7797356828193832

In [20]:
print(metrics.confusion_matrix(y_test,pred))

[[ 17   7   9]
 [  3  46  14]
 [  3  14 114]]


In [21]:
print(metrics.classification_report(y_test,pred,digits=3))

              precision    recall  f1-score   support

          -1      0.739     0.515     0.607        33
           0      0.687     0.730     0.708        63
           1      0.832     0.870     0.851       131

    accuracy                          0.780       227
   macro avg      0.753     0.705     0.722       227
weighted avg      0.778     0.780     0.776       227



In [22]:
def multiclass_roc_auc_score(y_test, pred, average="macro"):
    y_test = lb.transform(y_test)
    y_pred = lb.transform(pred)
    return roc_auc_score(y_test, pred, average=average)

### One vs rest Logistic Regression - Without Over-Sampling

In [23]:
# Create one-vs-rest logistic regression object
clf = LogisticRegression(random_state=0, multi_class='ovr')

In [24]:
model = clf.fit(X_train, y_train)

In [25]:
pred = model.predict(X_test)

In [26]:
from sklearn import metrics
score = metrics.accuracy_score(y_test, pred)
score

0.6563876651982379

In [27]:
print(metrics.confusion_matrix(y_test,pred))

[[  6   0  27]
 [  0  12  51]
 [  0   0 131]]


In [28]:
y.value_counts()

 1    393
 0    178
-1    114
Name: Label, dtype: int64

In [29]:
y_test.value_counts()

 1    131
 0     63
-1     33
Name: Label, dtype: int64

In [30]:
print(metrics.classification_report(y_test,pred,digits=3))

              precision    recall  f1-score   support

          -1      1.000     0.182     0.308        33
           0      1.000     0.190     0.320        63
           1      0.627     1.000     0.771       131

    accuracy                          0.656       227
   macro avg      0.876     0.457     0.466       227
weighted avg      0.785     0.656     0.578       227

